In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
import glob
import os
import shutil
import pandas as pd
from tqdm import tqdm_notebook

# Downloading CelebFaces Attributes (CelebA) Dataset from Kaggle

In [2]:
!pip install kaggle

In [0]:
!mkdir -p ~/.kaggle 
!echo '{"username":"miske01","key":"1fb5c9c5be6ead6c58e09d0a5e1526f1"}'> ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d jessicali9530/celeba-dataset

100% 1.32G/1.33G [00:31<00:00, 29.6MB/s]
100% 1.33G/1.33G [00:31<00:00, 45.1MB/s]


In [0]:
!unzip -I 'utf-8' 'celeba-dataset.zip' -d ' '

# Moving the images to the training and validation folders

In [0]:
os.mkdir('data/Faces')
os.mkdir('data/Faces/train')
os.mkdir('data/Faces/valid')

In [13]:
item_list = glob.glob('data/img_align_celeba/img_align_celeba/*.jpg')
print(len(item_list))

202599


In [0]:
path = 'data/Faces/'

In [15]:
for i in tqdm_notebook(range(len(item_list))):
  if i <= 178000:
    destination = path+'train/'
  else:
    destination = path+'valid/'
  try:
    shutil.move(item_list[i],destination+str(i).zfill(6)+'.jpg')
  except:
    pass

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


# Preparing the labels csv file

In [0]:
df_label = pd.read_csv('data/list_attr_celeba.csv')

In [21]:
df_label.head()

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,Blurry,Brown_Hair,Bushy_Eyebrows,Chubby,Double_Chin,Eyeglasses,Goatee,Gray_Hair,Heavy_Makeup,High_Cheekbones,Male,Mouth_Slightly_Open,Mustache,Narrow_Eyes,No_Beard,Oval_Face,Pale_Skin,Pointy_Nose,Receding_Hairline,Rosy_Cheeks,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,1,-1,1,-1,-1,1,-1,-1,1,-1,-1,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,-1,-1,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [0]:
columns = pd.Series(list(df_label.columns)[1:])

In [20]:
columns.head()

0    5_o_Clock_Shadow
1     Arched_Eyebrows
2          Attractive
3     Bags_Under_Eyes
4                Bald
dtype: object

In [0]:
def label_extract(raw):
  return(' '.join(columns[[True if i == 1 else False for i in raw[columns]]]))

In [0]:
df_label['label'] = df_label.apply(lambda x: label_extract(x),axis=1)

In [0]:
df_label = df_label.loc[:,['image_id','label']]

In [26]:
item_list = glob.glob('data/Faces/*/*.jpg')
item_df = pd.DataFrame({'image_name':pd.Series(item_list).apply(lambda x:x.split('/',2)[2])})
item_df['image_id'] = item_df.image_name.apply(lambda x: x.split('/',1)[1])
item_df.head()

,image_name,image_id
0,train/094351.jpg,094351.jpg
1,train/074285.jpg,074285.jpg
2,train/009891.jpg,009891.jpg
3,train/144725.jpg,144725.jpg
4,train/157346.jpg,157346.jpg


In [0]:
df_label = df_label.merge(item_df,on = 'image_id',how='inner')

In [28]:
df_label.head()

,image_id,label,image_name
0,000001.jpg,Arched_Eyebrows Attractive Brown_Hair Heavy_Ma...,train/000001.jpg
1,000002.jpg,Bags_Under_Eyes Big_Nose Brown_Hair High_Cheek...,train/000002.jpg
2,000003.jpg,Big_Lips Blurry Male Narrow_Eyes No_Beard Poin...,train/000003.jpg
3,000004.jpg,Attractive No_Beard Pointy_Nose Straight_Hair ...,train/000004.jpg
4,000005.jpg,Arched_Eyebrows Attractive Big_Lips Heavy_Make...,train/000005.jpg


In [0]:
df_label = df_label.loc[:,['image_name','label']]

In [30]:
df_label.head()

,image_name,label
0,train/000001.jpg,Arched_Eyebrows Attractive Brown_Hair Heavy_Ma...
1,train/000002.jpg,Bags_Under_Eyes Big_Nose Brown_Hair High_Cheek...
2,train/000003.jpg,Big_Lips Blurry Male Narrow_Eyes No_Beard Poin...
3,train/000004.jpg,Attractive No_Beard Pointy_Nose Straight_Hair ...
4,train/000005.jpg,Arched_Eyebrows Attractive Big_Lips Heavy_Make...


In [0]:
df_label.to_csv('data/Faces/labels.csv',index = False)